### Inverse Probability of Treatment Weighting (IPTW)

Motivating Example
- Supposed that there is a single binary confounder X

Suppose that P(A=1|X=1) = 0.1
- Among people with X = 1, only 10% will receive the treatment
- The value of the propensity score for people with X = 1 is 0.1

Supposed that P(A=1|X=0) = 0.8
- Among people with X = 0, 80% will receive treatment
- The value of the propensity score for people with X = 0 is 0.8

<img src="./img/iptw_start_1.png" >


### Treated Group (X = 1)

<img src="./img/iptw_start_2.png" >

__Propensity score matching:__
- Randomly choose 1 out of 9 control subjects to match with the only 1 treated subject.
- The control subject match is supposed to represent 9 others, but only uses information from itself.
- Thus, it can be seen as some form of discarding of information
<img src="./img/iptw_start_3.png" >

__IPTW:__   
- Rather than match, we can use all of the data but down-weight some and up-weight others.
    - You do not discard data in that sense from unmatched subjects.
- This is accomplished by weighting by the inverse of the probability of treatment received.
    - For treated subjects, weight by the inverse of P(A=1|X)
    - For control subjects, weight by the inverse of P(A=0|X)

<img src="./img/iptw_start_4.png" >

### Control Group (X = 0)

Propensity Score Matching for X = 0:
<img src="./img/iptw_start_5.png" >

IPTW for X = 0:
<img src="./img/iptw_start_6.png" >

#### Survey Sampling
- In surveys, it is common to oversample some groups relative to the population
    - Oversample a minority group
    - Oversample older adults
    - Oversample obese individuals
- To estimate the popualtion mean, we can weight the data to account for the oversample
    - Horvitz-Thompson estimator
    
This relates to observational studies.

In an observational study, certain groups are oversampled relative to the hypothetical sample from a randomized trial
- There is confounding in the original population
- IPTW creates a pseudo-population where treatment assignment no longer depends on X
    - No confounding in the pseudo population

Suppose P(A=1|X) = 0.9
- Apply weighting based on inverse probability weighing
- Create a pseudo population as if it was from a randomised control trial
<img src="./img/iptw_observational_1.png" >

- In the original population, some people were more likely to get treated than others based on their Xs.
- In the pseudo-population, everyone is equally likely to be treated regardless of their X values.

Under the assumption of exchangeability and positivity, we can estimate the __expected value of a potential outcome with treatment $E(Y^1)$__ as:

<img src="./img/iptw_formula_estimator.png" >

where $\pi_i = P(A=1|X_i)$ is the propensity score.

Note that the formula showcases how positivity assumption comes into play. If the propensity for a given X is 0, one of the values in the formula will be dividing by 0 which will result in a NaN number.

### Marginal Structural Models
- A marginal structural model (MSM) is a model for the mean of the potential outcomes
- __Marginal__: model that is not conditional on the confounders. we want to find the causal effect on the population, and not a subpopulation.
- __Structural__: Model for potential outcomes, not observed outcomes

#### __Linear MSM__

$E(Y^a) = \phi_0 + \phi_1 a,\ where\ a=0,1$

- $E(Y^0) = \phi_0$
- $E(Y^1) = \phi_0 + \phi_1$
- Thus, $\phi_1$ is the average causal effect $E(Y^1) - E(Y^0)$

#### __Logistic MSM (for binary outcome)__ 

$logit\{ E(Y^a) \} = \phi_0 + \phi_1 a,\ where \ a = 0,1$

So $exp(\phi_1)$ is the causal odds ratio

<img src="./img/msm_logistic_odds_ratio.png" >

#### __MSM with Effect Modification__
- MSMs can also include effect modifiers
- Suppose V is a variable that modifies the effect of A
- A linear MSM with effect modification:
    - This captures heterogeneity of treatment effect where the treatment effect might vary across different subpopulations

$E(Y^a|V) = \phi_0 + \phi_1 a + \phi_3 V + \phi_4 a V, \ where\ a = 0,1$

Thus, the causal effect for a subpopulation with V is:   

$E(Y^1|V) - E(Y^0|V) = \phi_1 + \phi_4 V$

#### __General MSM__

MSM models can be expanded to a general formula that is similar to that of generalised linear models with different link functions:

$g\{E(Y^a|V)\} = h(a,V;\phi)\ where $

- g() is a link function
- h() is a function specifying parametric form of a and V (typically additive, linear)


### Estimation in generalised linear regression models
- Estimation of parameters from a GLM:

$E(Y_i | X_i) = \mu_i = g^{-1}(X^T_i \beta)$

- Estimation involves solving 

$\sum^{n}_{i=1} \frac{\mathrm{d}\mu_i^T}{\mathrm{d} \beta} V_i^{-1} \{Y_i - \mu_i(\beta)\} = 0\ for\ \beta$

### Estimation in MSMs
- Marginal structural models look a lot like generalised linear models.
    - For example, $E(Y_i^a) = g^{-1} (\phi_0 + \phi_1 a)$
        - This involves "setting of treatment" and is applied to the whole population for potential outcome
- This model is __not equivalent (due to confounding)__ to the regression model
    - $E(Y_i|A_i) = g^{-1}(\phi_0 + \phi_1 A_i)$
        - This involves "conditioning of treatment" and is applied to only a subpopulation with possible confounding 

__Note the distinction between "Setting" and "Conditioning":__
- Setting is for MSMs
- Conditioning is for GLMs.
    - If you had a randomised trial, the parameters for the GLM regression model should represent a causal effect because there is no confounding.
    

- Recall that the __pseudo-population__ (obtained from IPTW) is free from confounding (assuming ignorability and positivity).
- By using the inverse propensity weights, we can apply those weights to the observed data and create a pseudo population that does not have confounding. 
- We can therefore estimate MSM parameters by solving estimating equations for the observed data of the __pseudo-population__.
  
  $\sum^{n}_{i=1} \frac{\mathrm{d}\mu_i^T}{\mathrm{d} \phi} V_i^{-1} W_i \{Y_i - \mu_i(\phi)\} = 0\ for\ \beta,\ where\ W_i = \frac{1}{A_iP(A=1|X_i)\ +\ (1-A_i)P(A=0|X_i)}$
  


### Steps:
1. Estimate propensity score (using a logistic regression model etc)
2. Create weights
    - 1 divided by propensity score for treated subjects
    - 1 divided by 1 minus the propensity score for control subjects
3. Specify the MSM of interest
4. Use software to fit a weighted generalised linear model
5. Use asymptotic (sandwich) variance estimator (or bootstrapping)
    - This accounts for the fact that pseudo-population might be larger than sample size

__Balance after weighting__
- Covariate balance can be checked on the weighted sample using standardized differences.
    - In a Table 1
    - In a plot

Recall the __standardised difference__: 
- It is the difference in means between groups, divided by the (pooled) standard deviation:
$SMD = \frac{\bar{X}_{treatment} - \bar{X}_{control}}{\sqrt{\frac{s^2_{treatment} + s^2_{control}}{2}}}$
- It was used for covariate balancing as shown in previously mentioned matching techniques

__SMD can be used on IPTW methods__
- Same idea, except on __weighted means and weighted variances__
    - Stratify on treatment group
        - Find weighted mean and weighted variance for each group
        - This can be done directly or with software tools that were developed for surveys (e.g. svydesign in R)
        - Take difference in weighted means and divide by an estimate of the pooled (weighted) standard deviation
        
__Example using RHC data with Weighted Pseudo-RCT-population__

<img src="./img/iptw_smd.png" >

<img src="./img/iptw_smd_plot.png" >

If there is covariate imbalance after weighting is already performed:
- Propensity score model should be refined with the following considerations:
    - Interactions
    - Non linearity

## Distribution of Weights

Issues with weights:
- Larger weights lead to noisier estimates of causal effects
- Example:
    - Suppose in a treatment group, 1 person has a weight of 10,000
    - If the outcome is binary (whether they have the event or not) could have a big impact on the parameter estimate
    - Essentially, if one person's outcome data can greatly affect the parameter estimate, then the __standard error will be large__.
    
Further intuition with __Bootstrapping__:
1. Randomly sample with replacement from the original sample
2. Estimate parameters
3. Repeat steps 1 and 2 many times.
4. The standard deviation of the bootstrap estimates is an estimate of the standard error.

Implications of Bootstrap results
- Someone with a very large weight will be included in some bootstrap samples (with possibility of repetition), but not others.
- Whether or not they are included will have a relatively large impact on the parameter estimates
- Thus, a lot of the variability of the estimator would be due to this one subject.

Relationship with Positivity Assumption:
- An extremely large weight means that the proability of that treatment was very small.
    - Thus, large weights indicate near violations of the positivity assumption
        - People with certain values of the covariates are very unlikely to get one of the treatments.

__Weights Distribution Plot for Diagnosis:__

<img src="./img/iptw_weights_distribution.png" >

__Summary statistics of weights:__

<img src="./img/iptw_weights_table.png" >

### Large Weights

A good first step is to look into why the weights are large. Identify the subjects who ahve large weights:
- What is unusual about them?
- Is there a problem with their data?
- Is there a problem with the propensity score model?

Very large weights: investigative step
- Example: suppose there is one confounder and you fit a logistic regression propensity score model:
$logit(\pi_i) = \beta_0 + \beta_1 X_i$

<img src="./img/iptw_large_weights_1.png" >

<img src="./img/iptw_large_weights_2.png" >

- For data points in the huge cluster, the propensity model of the logit is probably accurate within that region.
- For the outlier, it is too far out to the extreme with no other data points. Thus, the extrapolation of the logit curve might not actually be accurate.
    - Note the plateau of the curve, which was extrapolated without good support in the extreme end.
    - For example, an alternative red curve for the logit curve could be here:
    <img src="./img/iptw_large_weights_3.png" >

### Trimming the tails
- Large weights occur at observations in the __tails of the propensity score distribution__
- Trimming the tails can eliminate some of the extreme weights
    - This means __removing subjects__ who have __extreme values__ of the propensity score
        - A common trimming straegy:
            - Remove treated subjects whose propensity scores are above the 98th percentile from the distribution among __controls__
            - Remove control subjects whose propensity scores are below the 2nd percentile from the distribution of __treated subjects__
- __Reminder: trimming the tails changes the population!!!__

### Weight Truncation

Truncating the large weights is another option. Steps:
- Determine a maximum allowable weight
    - Can be a specific value
    - Can be based on a percentile
- If a weight is greater than the mx allowable, set it to the maximum allowable value
    - If your limit is 100 and someone has a weight of 2000, set theri weight to 100 instead.

Whether or not to truncate weights involves a bias-variance trade-off:
- __Truncation__: bias, but smaller variance
- __No truncation__: unbiased, larger variance

Truncating extremely large weights can result in estimators with lower mean squared error (MSE).

## Doubly Robust Estimation (aka Augmented Inverse Probabiliyt of Treatment Weighting)

__Method 1: Potential Outcome Estimation with IPTW__
- We can estimate $E(Y^1)$ using IPTW  
    $\frac{1}{n}\sum{n}{i=1} \frac{A_i Y_i}{\pi_i(X_i)},\ where\ \pi_i(X_i)\ are\ the\ inverse\ weights$
    
- If the propensity score is correctly specified, this estimator is unbiased.

__Method 2: Regression-based estimation__
- Alternatively, we can estimate $E(Y^1)$ by specifying an outcome regression model $m_1(X) = E(Y|A=1,X)$ and then averaging over the distribution of X.
    - NOTE that $m_1(X)$ is the regression model for outcome with Treatment A = 1
- If outcome model is correctly specified, then this estimator is unbiased.

<img src="./img/doubly_robust_estimate_regression_1.png" >

Key Breakdown:
- For treated subjects, we will definitely use their observed outcome Y
- For non-treated subjects, we will __predict their value of outcome Y given their X IF they had received treatment (A = 1)__.

__Doubly Robust Estimator__ is an estimator that is unbiased if __either the pronesity score model OR the outome regression model are correctly specified__. An example of this is:

<img src="./img/doubly_robust_estimate_formula_1.png" >

The doubly robust estimator is made of two components:
- IPTW: Dependent on the propensity scoring model through the weights.
- Augmentation: Dependent on the outcome regression model

__If propensity score is correctly specified, but outcome model is not__:
- Recall that $A_i$ is the treatment applied, while $\pi_i(X_i)$ is the propensity score.
- Expectation of $A_i$ is equal to the propensity score if propensity score model is correctly specified.
- __Weight of the Augmentation part has expectation of 0 based on the numerator.__
- This will leave us with only the IPTW part of the formula.

<img src="./img/doubly_robust_estimate_breakdown_1.png" >

__If propensity score is wrong, but outcome model is correct__:
- Outcome model is correct implies that the expectation of Y conditional on X should be equal to $m_1(X_i)$
- The above formula can be re-written into the following:

<img src="./img/doubly_robust_estimate_breakdown_2.png" >
 
- Given that the outcome model is correct, the numerator portion $Y_i - m_1(X_i)$ is expected to be 0.
- This only leaves the 2nd portion of the revised formula with only $m_1(X_i)$ which is the expected value of $Y^1$.

Doubly robust estimators are also known as augmented IPTW (AIPTW) estimators.
- Can use semi parametric theory to identify best estimators
- In general, AIPTW estimators should be __more efficient__ than regular IPTW estimators.
    - This means they have a smaller variance associated with them.

# IPTW Workflow in R


- __RHC dataset:__
    - Treatment: RHC or not
    - Outcome: Death (yes/no)
    - Confounders as Covariates X: x1, x2, x3
- __Necessary packages__
    - `library(tableone), library(ipw), library(sandwich), library(survey)`
- __Propensity score__
    - Create propensity score model with logistic regression
        - `psmodel <- glm(treatment ~ x1 + x2 + x3, family = binomial(link = "logit"))`
        - `ps <- predict(psmodel, type = "response")`
    - Check plot of propensity score (pre-matching)
        - <img src="./img/doubly_robust_estimate_workflow_1.png" >
- __IPTW (Part 1): Weights__
    - Create weights and check balance (SMD) for the pseudo population
        - `weight <- ifelse(treatment==1, 1/ps, 1/(1-ps))`
        - `weighteddata <- svydesign(ids =~ 1, data = mydata, weights = ~ weights)`
        - `weightedtable <- svyCreateTableOne(vars = xvars, strata = "treatment", data = weighteddata, test = FALSE)`
- __IPTW (Part 2): MSMs__
    - Fit the following MSM using IPTW
        - $E(Y^a_i) = g^{-1}(\phi_0 + \phi_1 a)$
            - A is treatment
            - Y is death (yes, no)
            - g() is the link function
                - Log link for causal relative risk
                - Identity link for causal risk difference
    - Code (Relative Risk):
        - Create MSM model through regression since it is a pseudo balanced (no confounding) population 
            - `glm.obj <- glm(died ~ treatment, weights = weight, family = binomial(link = log))`
        - Check summary
            - `betaiptw <- coef(glm.obj)`
        - To properly account for weighting
            - `SE <- srqt(diag(vcovHC(glm.obj, type = "HC0")))`
        - Extract out point estimates and CI for relative risk
            - `causalrr <- exp(betaiptw[2])`
            - `lcl <- exp(betaiptw[2] - 1.96*SE[2])`
            - `ucl <- exp(betaiptw[2] + 1.96*SE[2])`
            - `c(lcl, causalrr, ucl)`
    - Code (Causal Risk Difference):
        - Create MSM model through regression since it is a pseudo balanced (no confounding) population 
            - `glm.obj <- glm(died ~ treatment, weights = weight, family = binomial(link = "identity"))`
        - Check summary
            - `betaiptw <- coef(glm.obj)`
        - To properly account for weighting
            - `SE <- srqt(diag(vcovHC(glm.obj, type = "HC0")))`
        - Extract out point estimates and CI for causal risk
            - `causalrd <- exp(betaiptw[2])`
            - `lcl <- exp(betaiptw[2] - 1.96*SE[2])`
            - `ucl <- exp(betaiptw[2] + 1.96*SE[2])`
            - `c(lcl, causalrd, ucl)`
        


__Alternative method with IPW package__
- __Propensity Scoring:__
    - Create propensity score model with logistic regression
        - `weightmodel <- ipwpoint(exposure = treatment, family = "binomial", link = "logit", denominator ~ x1 + x2 + x3, data = mydata)
        - `summary(weightmodel$ipw.weights)`
    - Check plot of propensity score (pre-matching)
        - `ipwplot(weights = weightmodel$ipw.weights, logscale = FALSE, main =  "weights", xlim = c(0, 22))`
            - <img src="./img/doubly_robust_estimate_workflow_2.png" >
- __Fit the MSM__
    - Create MSM model through regression since it is a pseudo balanced (no confounding) population. svyglm function will do the robust sandwich estimator automatically
        - `msm <- (svyglm(died ~ treatment, design = svydesign(~1, weights = ~wt, data = mydata)))`
        - `coef(msm)`
        - `confint(msm)`

__Truncating weights:__
- __Non-IPW package:__
    -  `truncweight <- replace(weight, weight > 10, 10)`
    - `glm.obj <- glm(died~treatment, weights = truncweight, family = binomial(link = "identity"))`
- __IPW package:__
    - Truncation using percentile on both ends (ie. 0.01 implies 1st and 99th percentiles)
        - `weightmodel <- ipwpoint(exposure = treatment, family = "binomial", link = "logit", denominator = ~ x1 + x2 + x3, data = mydata, trunc = 0.01)`
    